# Explore Company Records

**Author:** Hai Ran Zheng

**Date:** April 18th, 2025

**Description:** Look into possible Data Quality issues for web crawled data 

## Set Up Packages and Paths

In [235]:
import sys
import os
root_path = os.path.dirname(os.path.abspath(os.curdir))
sys.path.insert(0, root_path)
dataset_path = os.path.join(root_path, "output")

In [236]:
# Load DataFrame
import polars as pl
df = pl.read_ndjson(os.path.join(dataset_path, "company_records.jsonl"))

## Titles that don't start with X

In [237]:
len(df)

216

In [238]:
len(df.filter(
    pl.col("TITLE").list.get(0).str.to_lowercase().str.starts_with("x")
))


196

Around 20 companies whose title doesn't start with x, but is returned from query on the webapp

In [104]:
df.filter(
    ~pl.col("TITLE").list.get(0).str.to_lowercase().str.starts_with("x")
).head()

SORT_INDEX,TITLE,ID,FILING_DATE,RECORD_NUM,STATUS,STANDING,ALERT,CAN_REINSTATE,CAN_FILE_AR,CAN_ALWAYS_FILE_AR,CAN_FILE_REINSTATEMENT,ID_key,DRAWER_DETAIL_LIST
i64,list[str],i64,str,str,str,str,bool,bool,bool,bool,bool,str,list[struct[5]]
0,"[""Accurate INSPEX, LLC"", ""Limited Liability Company - Business - Foreign""]",171478,"""09/24/2018""","""0000171856""","""Active""","""Good Standing""",false,false,false,false,false,"""171478""","[{""Filing Type"",""Limited Liability Company - Business - Foreign"",null,null,false}, {""Status"",""Active"",null,null,false}, … {""Commercial Registered Agent"",""COGENCY GLOBAL INC. 120 W SWEET AVE STE 9 BISMARCK, ND 58504"",null,null,false}]"
19,"[""RXO Last Mile, Inc."", ""Corporation - Business - Foreign""]",90563,"""01/31/2008""","""0000090931""","""Active""","""Good Standing""",false,false,true,true,false,"""90563""","[{""Filing Type"",""Corporation - Business - Foreign"",null,null,false}, {""Status"",""Active"",null,null,false}, … {""Commercial Registered Agent"",""REGISTERED AGENT SOLUTIONS, INC. 1709 N 19TH ST STE 3 BISMARCK, ND 58501-2121"",null,null,false}]"
16,"[""Miller Underground, LLC"", ""Limited Liability Company - Business - Domestic""]",153110,"""04/25/2016""","""0000153484""","""Active""","""Good Standing""",false,false,false,false,false,"""153110""","[{""Filing Type"",""Limited Liability Company - Business - Domestic"",null,null,false}, {""Status"",""Active"",null,null,false}, … {""Registered Agent"",""JARED MILLER 701 SUNFLOWER DR BISMARCK, ND 58503"",null,null,false}]"
15,"[""MCKESSON MEDICAL-SURGICAL MEDIMART INC."", ""Corporation - Business - Foreign""]",31989,"""04/22/1991""","""0000032178""","""Active""","""Good Standing""",false,false,true,true,false,"""31989""","[{""Filing Type"",""Corporation - Business - Foreign"",null,null,false}, {""Status"",""Active"",null,null,false}, … {""Commercial Registered Agent"",""CORPORATION SERVICE COMPANY 418 N 2ND ST BISMARCK, ND 58501"",null,null,false}]"
18,"[""RXO Corporate Solutions, LLC"", ""Limited Liability Company - Business - Foreign""]",337397,"""06/10/2022""","""0005914988""","""Active""","""Good Standing""",false,false,false,false,false,"""337397""","[{""Filing Type"",""Limited Liability Company - Business - Foreign"",null,null,false}, {""Status"",""Active"",null,null,false}, … {""Commercial Registered Agent"",""REGISTERED AGENT SOLUTIONS, INC. 1709 N 19TH ST STE 3 BISMARCK, ND 58501-2121"",null,null,false}]"


## Remove Duplicates

In [ ]:
# Since title is stored in array, let's move them to their individual dataframe columns
df = df.with_columns(
    COMPANY_TITLE=pl.col("TITLE").list[0],
    COMPANY_TYPE=pl.col("TITLE").list[1]
)

In [107]:
duplicated_company_titles = df.filter(pl.col("COMPANY_TITLE").is_duplicated())

In [108]:
duplicated_company_titles.sort("COMPANY_TITLE").head(10)

SORT_INDEX,TITLE,ID,FILING_DATE,RECORD_NUM,STATUS,STANDING,ALERT,CAN_REINSTATE,CAN_FILE_AR,CAN_ALWAYS_FILE_AR,CAN_FILE_REINSTATEMENT,ID_key,DRAWER_DETAIL_LIST,COMPANY_TITLE,COMPANY_TYPE
i64,list[str],i64,str,str,str,str,bool,bool,bool,bool,bool,str,list[struct[5]],str,str


In [109]:
len(duplicated_company_titles.unique("COMPANY_TITLE"))

0

No COMPANY_TITLE duplicates found, good

## Investigate Missing Graph Relationships

From logs, I noticed 2 businesses were not capturing any graph relationships from our expected columns: OWNER_NAME, COMMERCIAL_REGISTERED_AGENT, OR REGISTERED_AGENT. I discovered the a new field OWNERS need to be captured, let's see how this column works.

We need to move DRAWER_DETAIL_LIST into our columns first

In [110]:
expanded = (
    df.explode("DRAWER_DETAIL_LIST")
      .unnest("DRAWER_DETAIL_LIST")
)

In [111]:
expanded.head()

SORT_INDEX,TITLE,ID,FILING_DATE,RECORD_NUM,STATUS,STANDING,ALERT,CAN_REINSTATE,CAN_FILE_AR,CAN_ALWAYS_FILE_AR,CAN_FILE_REINSTATEMENT,ID_key,LABEL,VALUE,TYPE,LINKLABEL,ALERT_YN,COMPANY_TITLE,COMPANY_TYPE
i64,list[str],i64,str,str,str,str,bool,bool,bool,bool,bool,str,str,str,null,null,bool,str,str
0,"[""Accurate INSPEX, LLC"", ""Limited Liability Company - Business - Foreign""]",171478,"""09/24/2018""","""0000171856""","""Active""","""Good Standing""",false,false,false,false,false,"""171478""","""Filing Type""","""Limited Liability Company - Bu…",null,null,false,"""Accurate INSPEX, LLC""","""Limited Liability Company - Bu…"
0,"[""Accurate INSPEX, LLC"", ""Limited Liability Company - Business - Foreign""]",171478,"""09/24/2018""","""0000171856""","""Active""","""Good Standing""",false,false,false,false,false,"""171478""","""Status""","""Active""",null,null,false,"""Accurate INSPEX, LLC""","""Limited Liability Company - Bu…"
0,"[""Accurate INSPEX, LLC"", ""Limited Liability Company - Business - Foreign""]",171478,"""09/24/2018""","""0000171856""","""Active""","""Good Standing""",false,false,false,false,false,"""171478""","""Standing - AR""","""Good""",null,null,false,"""Accurate INSPEX, LLC""","""Limited Liability Company - Bu…"
0,"[""Accurate INSPEX, LLC"", ""Limited Liability Company - Business - Foreign""]",171478,"""09/24/2018""","""0000171856""","""Active""","""Good Standing""",false,false,false,false,false,"""171478""","""Standing - RA""","""Good""",null,null,false,"""Accurate INSPEX, LLC""","""Limited Liability Company - Bu…"
0,"[""Accurate INSPEX, LLC"", ""Limited Liability Company - Business - Foreign""]",171478,"""09/24/2018""","""0000171856""","""Active""","""Good Standing""",false,false,false,false,false,"""171478""","""Standing - Other""","""Good""",null,null,false,"""Accurate INSPEX, LLC""","""Limited Liability Company - Bu…"


In [112]:
businesses_expanded_details = expanded.pivot(
    "LABEL",
    values="VALUE",
    index="TITLE"
)

In [139]:
businesses_expanded_details.head()

TITLE,Filing Type,Status,Standing - AR,Standing - RA,Standing - Other,Formed In,Term of Duration,Initial Filing Date,Principal Address,Mailing Address,AR Due Date,Commercial Registered Agent,Registered Agent,Owner Name,Owner Address,Nature of Business,Expiration Date,Owners,,Delayed Effective Date,Filing Subtype
list[str],str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"[""Accurate INSPEX, LLC"", ""Limited Liability Company - Business - Foreign""]","""Limited Liability Company - Bu…","""Active""","""Good""","""Good""","""Good""","""INDIANA""","""Perpetual""","""09/24/2018""","""6000 FREEDOM SQUARE DRIVE SUIT…","""6000 FREEDOM SQUARE DR STE 300…","""11/17/2025""","""COGENCY GLOBAL INC. 120 W SWEE…",null,null,null,null,null,null,null,null,null
"[""XCALIBER INTERNATIONAL LTD., L.L.C."", ""Limited Liability Company - Business - Foreign""]","""Limited Liability Company - Bu…","""Active""","""Good""","""Good""","""Good""","""OKLAHOMA""","""Perpetual""","""02/01/2016""","""1 TOBACCO RD PRYOR, OK 74361-4…","""4747 NE 1ST ST PRYOR, OK 74361…","""11/17/2025""","""CORPORATION SERVICE COMPANY 41…",null,null,null,null,null,null,null,null,null
"[""Xavier Collections LLC"", ""Limited Liability Company - Business - Domestic""]","""Limited Liability Company - Bu…","""Active""","""Good""","""Good""","""Good""","""NORTH DAKOTA""","""Perpetual""","""03/06/2024""","""835 34TH AVE E APT 201 WEST FA…","""PO BOX 10301 FARGO, ND 58106-0…","""11/17/2025""",null,"""KORTO C KOLLIE 835 34TH AVE E …",null,null,null,null,null,null,null,null
"[""Xavier Taylor Construction"", ""Trade Name""]","""Trade Name""","""Active""",null,null,null,null,"""Expires - 5 years""","""06/27/2023""","""4805 141 ST AVE NW WILLISTON, …","""4805 141ST AVE NW WILLISTON, N…",null,null,null,"""Xavier G Taylor""","""4805 141ST AVE NW WILLISTON, N…","""Various types of construction …","""06/27/2028""",null,null,null,null
"[""XAPIC STUDIOS"", ""Trade Name""]","""Trade Name""","""Active""",null,null,null,null,"""Expires - 5 years""","""10/14/2016""","""607 N 3RD ST # 1 GRAND FORKS, …","""607 N 3RD ST # 1 GRAND FORKS, …",null,null,null,"""PATRICIA A ROGALLA""","""607 N 3RD ST # 1 GRAND FORKS, …","""I attend Vendor shows and offe…","""10/14/2026""",null,null,null,null


Now we can finally look at how Owners is structured

## 

In [114]:
businesses_expanded_details.select(
    pl.col("Owners")
).head()

Owners
str
null
null
null
null
null


In [115]:
businesses_expanded_details.filter(
    pl.col("Owners").is_not_null()
)

TITLE,Filing Type,Status,Standing - AR,Standing - RA,Standing - Other,Formed In,Term of Duration,Initial Filing Date,Principal Address,Mailing Address,AR Due Date,Commercial Registered Agent,Registered Agent,Owner Name,Owner Address,Nature of Business,Expiration Date,Owners,,Delayed Effective Date,Filing Subtype
list[str],str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"[""XTREME SIGNS & GRAPHIX"", ""Trade Name""]","""Trade Name""","""Active""",null,null,null,null,"""Expires - 5 years""","""06/13/2006""","""401 6TH ST SE DEVILS LAKE, ND …","""401 6TH ST SE DEVILS LAKE, ND …",null,null,null,null,null,"""SIGN DESIGN, FABRICATION OF SI…","""06/13/2026""","""CRAIG NICHOLAS THOMAS 8482 HIG…","""AMANDA KAY THOMAS 8482 HIGHWAY…",null,null
"[""XL Farms"", ""Trade Name""]","""Trade Name""","""Active""",null,null,null,null,"""Expires - 5 years""","""03/04/2020""","""14079 41ST ST NW ALEXANDER, ND…","""14079 41ST ST NW ALEXANDER, ND…",null,null,null,null,null,"""Farm trucking""","""03/04/2030""","""Corey K Getzlaff 14079 41ST ST…","""Cindy J Getzlaff 14079 41ST ST…",null,null


In the drawer details array of dicts, dicts with "Owners" labels are typically followed by dicts with empty string labels, which by design, allows API data to return multiple owners.


## Check Company Title

In [119]:
df.select(
    pl.col("COMPANY_TITLE")
)

COMPANY_TITLE
str
"""Accurate INSPEX, LLC"""
"""XCALIBER INTERNATIONAL LTD., L…"
"""Xavier Collections LLC"""
"""Xavier Taylor Construction"""
"""XAPIC STUDIOS"""
…
"""CONDUENT HR SERVICES, LLC"""
"""AKITA EQUIPMENT CORP."""
"""ATOS IT OUTSOURCING SERVICES, …"


In [120]:
df.select(
    pl.col("COMPANY_TITLE")
).unique()

COMPANY_TITLE
str
"""XCMG America Financial LLC"""
"""XCS Pros, LLC"""
"""CONDUENT BUSINESS SERVICES, LL…"
"""Xquisite Construction LLC"""
"""XCELL DRIVING SCHOOL LLP"""
…
"""X-TREME OUTFITTERS"""
"""X-treme Detailing"""
"""XSM ATM Services LLC"""


In [138]:
df.select(
    (pl.col("COMPANY_TITLE").str.to_uppercase() == pl.col("COMPANY_TITLE")).alias("IS_ALL_UPPERCASE")
).group_by("IS_ALL_UPPERCASE").len()

IS_ALL_UPPERCASE,len
bool,u32
false,115
true,101


Half company titles are upper case, half are not.

## Check Relationships

interested_labels = (
            "COMMERCIAL_REGISTERED_AGENT",
            "REGISTERED_AGENT",
            "OWNER_NAME",
            "OWNERS",
        )

In [196]:
interested_labels = (
    "Commercial Registered Agent",
    "Registered Agent",
    "Owner Name",
    "Owners",
    ""
)

In [197]:
# businesses_expanded_details.columns = column.upper for column businesses_expanded_details.columns

In [198]:
businesses_graph_details = businesses_expanded_details.select(
    interested_labels
)

In [199]:
businesses_graph_details.head()

Commercial Registered Agent,Registered Agent,Owner Name,Owners,
str,str,str,str,str
"""COGENCY GLOBAL INC. 120 W SWEE…",null,null,null,null
"""CORPORATION SERVICE COMPANY 41…",null,null,null,null
null,"""KORTO C KOLLIE 835 34TH AVE E …",null,null,null
null,null,"""Xavier G Taylor""",null,null
null,null,"""PATRICIA A ROGALLA""",null,null


In [203]:
stacked = businesses_graph_details.melt().select("value")

C:\Users\harri\AppData\Local\Temp\ipykernel_11476\1333292206.py:1: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  stacked = businesses_graph_details.melt().select("value")


In [201]:
len(businesses_graph_details)

216

In [205]:
assert len(businesses_graph_details) * 5 == len(stacked)

In [206]:
stacked = stacked.filter(
    pl.col("value").is_not_null()
)

In [207]:
stacked.head()

value
str
"""COGENCY GLOBAL INC. 120 W SWEE…"
"""CORPORATION SERVICE COMPANY 41…"
"""CORPORATION SERVICE COMPANY 41…"
"""Registered Agents Inc 3003 32N…"
"""CORPORATION SERVICE COMPANY 41…"


I don't want to analysis the columns individually for now, I just want to get the overall vibe of their format. If \n exist all the time or not.

In [208]:
stacked.select(
    pl.col("value").str.contains("\n").alias("has_n")
).group_by("has_n").len()

has_n,len
bool,u32
true,177
false,41


In [209]:
arr = stacked.filter(
    ~pl.col("value").str.contains("\n").alias("has_n")
).to_numpy().tolist()

In [210]:
arr[10:20]

[['Xiomara A. Uribe, Ecaterina M. Quintana'],
 ['Scott A Kraft'],
 ['Travis c Toepke'],
 ['LIFE LINE TECHNOLOGIES, LLC'],
 ['KARLGAARD ENTERPRISES, LLC'],
 ['Dustin R Smith'],
 ['Christopher Tegwe'],
 ['MONEYDART GLOBAL SERVICES INC.'],
 ['XPT Partners, LLC'],
 ['JEROME M HINDS']]

Xiomara A. Uribe, Ecaterina M. Quintana. If you search their name up in data lake output/company_records.jsonl, it's very interesting. Their data is stored under OWNER_NAME when there's actually two owner names.

### Deeper Dive into Relationships

In [211]:
def slash_n_counts(df : pl.DataFrame, column):
    return df.select(
        pl.col(column).str.contains("\n").alias(f"has_n_{column}")
    ).group_by(f"has_n_{column}").len()

def filter_nulls(df : pl.DataFrame, column):
    return df.filter(
        pl.col(column).is_not_null()
    ).select(column)

In [212]:
businesses_graph_details.head()

Commercial Registered Agent,Registered Agent,Owner Name,Owners,
str,str,str,str,str
"""COGENCY GLOBAL INC. 120 W SWEE…",null,null,null,null
"""CORPORATION SERVICE COMPANY 41…",null,null,null,null
null,"""KORTO C KOLLIE 835 34TH AVE E …",null,null,null
null,null,"""Xavier G Taylor""",null,null
null,null,"""PATRICIA A ROGALLA""",null,null


In [213]:
businesses_graph_details.columns

['Commercial Registered Agent', 'Registered Agent', 'Owner Name', 'Owners', '']

### Commercial Registered Agents

In [214]:
cra = filter_nulls(businesses_graph_details, "Commercial Registered Agent")
slash_n_counts(cra, "Commercial Registered Agent")

has_n_Commercial Registered Agent,len
bool,u32
true,103


In [215]:
cra.to_numpy().tolist()[30:40]

[['Montgomery & Pender, P.C.\n5630 34TH AVE S\nSTE 120\nFARGO, ND  58104'],
 ['LEGALINC CORPORATE SERVICES INC.\n2905 JUDE LN NW\nMANDAN, ND  58554'],
 ['C T CORPORATION SYSTEM\n120 W SWEET AVE\nBISMARCK, ND  58504'],
 ['REGISTERED AGENT SOLUTIONS, INC.\n1709 N 19TH ST STE 3\nBISMARCK, ND  58501-2121'],
 ['CORPORATION SERVICE COMPANY\n418 N 2ND ST\nBISMARCK, ND  58501'],
 ['C T CORPORATION SYSTEM\n120 W SWEET AVE\nBISMARCK, ND  58504'],
 ['CORPORATION SERVICE COMPANY\n418 N 2ND ST\nBISMARCK, ND  58501'],
 ['CORPORATION SERVICE COMPANY\n418 N 2ND ST\nBISMARCK, ND  58501'],
 ['CORPORATION SERVICE COMPANY\n418 N 2ND ST\nBISMARCK, ND  58501'],
 ['NORTHWEST REGISTERED AGENT SERVICE, INC.\n3003 32ND AVE S STE 240\nFARGO, ND  58103-6118']]

### Registered Agents

In [ ]:
cra = filter_nulls(businesses_graph_details, "Registered Agent")
slash_n_counts(cra, "Registered Agent")

has_n_Registered Agent,len
bool,u32
true,70


In [ ]:
cra.to_numpy().tolist()[60:70]

[['TAIT  SILTALA\n722 9TH AVE W\nWILLISTON, ND  58801'],
 ['CHRISTOPHER  PERLEBERG\n710 17TH ST NW\nJAMESTOWN, ND  58401-2241'],
 ['ANTHONY  BAUMGARTNER\n1292 56TH ST SE\nBRADDOCK, ND  58524-9244'],
 ['GREGORY C KRUTSINGER\n650 2ND AVE NW\nWEST FARGO, ND  58078'],
 ['TAIT  SILTALA\n722 9TH AVE W\nWILLISTON, ND  58801'],
 ['MIAN  HUSSAIN\n3017 31ST AVE W\nAPT 305B\nWILLISTON, ND  58801'],
 ['ERIC  CHRISTIANS\n3749 14TH ST W\nWEST FARGO, ND  58078'],
 ['DANNY  KRAFT\n325 7TH ST S\nSTE 100\nFARGO, ND  58103'],
 ['KENDALL  VETTER\n2341 BAYVIEW COURT SE\nMANDAN, ND  58554'],
 ['JARED  MILLER\n701 SUNFLOWER DR\nBISMARCK, ND  58503']]

### Owner Name

In [240]:
cra = filter_nulls(businesses_graph_details, "Owner Name")
slash_n_counts(cra, "Owner Name")

has_n_Owner Name,len
bool,u32
false,41


In [245]:
cra.to_numpy().tolist()[10:20]

[['Xiomara A. Uribe, Ecaterina M. Quintana'],
 ['Scott A Kraft'],
 ['Travis c Toepke'],
 ['LIFE LINE TECHNOLOGIES, LLC'],
 ['KARLGAARD ENTERPRISES, LLC'],
 ['Dustin R Smith'],
 ['Christopher Tegwe'],
 ['MONEYDART GLOBAL SERVICES INC.'],
 ['XPT Partners, LLC'],
 ['JEROME M HINDS']]

### Owners

In [220]:
cra = filter_nulls(businesses_graph_details, "Owners")
slash_n_counts(cra, "Owners")

has_n_Owners,len
bool,u32
true,2


In [222]:
cra.to_numpy().tolist()

[['CRAIG NICHOLAS THOMAS\n8482 HIGHWAY 2\nDEVILS LAKE, ND 58301-8742'],
 ['Corey K Getzlaff\n14079 41ST ST NW\nALEXANDER, ND 58831-8719']]

### Owner

In [224]:
cra = filter_nulls(businesses_graph_details, "")
slash_n_counts(cra, "")

has_n_,len
bool,u32
true,2


In [226]:
cra.to_numpy().tolist()

[['AMANDA KAY THOMAS\n8482 HIGHWAY 2\nDEVILS LAKE, ND 58301-8742'],
 ['Cindy J Getzlaff\n14079 41ST ST NW\nALEXANDER, ND 58831-8719']]

So far, Xiomara A. Uribe, Ecaterina M. Quintana appears to be the only data quality issue. 

In [228]:
stacked.filter(
    pl.col("value") == "Xiomara A. Uribe, Ecaterina M. Quintana"
)

value
str
"""Xiomara A. Uribe, Ecaterina M.…"


In [230]:
df.filter(
    pl.col("COMPANY_TITLE").str.to_lowercase() == "Xiomara A. Uribe, Ecaterina M. Quintana"
)

SORT_INDEX,TITLE,ID,FILING_DATE,RECORD_NUM,STATUS,STANDING,ALERT,CAN_REINSTATE,CAN_FILE_AR,CAN_ALWAYS_FILE_AR,CAN_FILE_REINSTATEMENT,ID_key,DRAWER_DETAIL_LIST,COMPANY_TITLE,COMPANY_TYPE
i64,list[str],i64,str,str,str,str,bool,bool,bool,bool,bool,str,list[struct[5]],str,str


Probably shouldn't take action until further questions. People can have identify themselves with two names after all.

In [233]:
businesses_expanded_details.filter(
    pl.col("Owner Name") == "Xiomara A. Uribe, Ecaterina M. Quintana"
)

TITLE,Filing Type,Status,Standing - AR,Standing - RA,Standing - Other,Formed In,Term of Duration,Initial Filing Date,Principal Address,Mailing Address,AR Due Date,Commercial Registered Agent,Registered Agent,Owner Name,Owner Address,Nature of Business,Expiration Date,Owners,,Delayed Effective Date,Filing Subtype
list[str],str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"[""Xtravagant Elegance"", ""Trade Name""]","""Trade Name""","""Active""",null,null,null,null,"""Expires - 5 years""","""04/05/2022""","""1002 EAST LASALLE DR. BISMARCK…","""1002 E LASALLE DR BISMARCK, ND…",null,null,null,"""Xiomara A. Uribe, Ecaterina M.…","""1002 E LASALLE DR BISMARCK, ND…","""luxury balloon decor for all e…","""04/05/2027""",null,null,null,null


In [234]:
df.filter(
    pl.col("COMPANY_TITLE") == "Xtravagant Elegance"
)

SORT_INDEX,TITLE,ID,FILING_DATE,RECORD_NUM,STATUS,STANDING,ALERT,CAN_REINSTATE,CAN_FILE_AR,CAN_ALWAYS_FILE_AR,CAN_FILE_REINSTATEMENT,ID_key,DRAWER_DETAIL_LIST,COMPANY_TITLE,COMPANY_TYPE
i64,list[str],i64,str,str,str,str,bool,bool,bool,bool,bool,str,list[struct[5]],str,str
182,"[""Xtravagant Elegance"", ""Trade Name""]",333362,"""04/05/2022""","""0005852807""","""Active""","""Good Standing""",false,false,false,false,false,"""333362""","[{""Filing Type"",""Trade Name"",null,null,false}, {""Status"",""Active"",null,null,false}, … {""Mailing Address"",""1002 E LASALLE DR BISMARCK, ND 58503-8895"",null,null,false}]","""Xtravagant Elegance""","""Trade Name"""


Xiomara A. Uribe, Ecaterina M. Quintana is the Owner Nameof Xtravagant Elegance, SOS CONTROL ID# 5852807